### Basic trading

- entry: do I use only the data for that stock endogenous, or also include the data from other stocks, exagenous.
- entry and exit :
- exit : take profit, stop loss, time out

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf  # for data
from pandas_datareader import data as pdr

In [2]:
yf.pdr_override()

In [3]:
stock = input('Enter stock tikcer symbol:')

Enter stock tikcer symbol: BB


In [4]:
stock

'BB'

In [5]:
start_year = 2018
start_month = 1
start_day = 1

start = dt.datetime(start_year,start_month,start_day)
now = dt.datetime.now()
df = pdr.get_data_yahoo(stock,period = "35d",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "5m",)


emasUsed = [26,50]
for ema in emasUsed:
    df['Ema_' + str(ema)] = round(df['Adj Close'].ewm(span = ema, adjust = False).mean(),2)
    df['Middle Band'] =df['Adj Close'].rolling(window=20).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Close'] > df['Upper Band'],'above_ballinger','normal')
    
df = df.iloc[20:,:]


[*********************100%***********************]  1 of 1 completed


In [ ]:
#### 1. STRATEGY CROSSING

In [ ]:
pos = 0
num = 0
percentchange = []

for i in df.index:
    cmin = df['Ema_26'][i]
    cmax = df['Ema_50'][i]
    status_lower = df['status_lower'][i]
    
    close = df['Adj Close'][i]
    
    #if(status_lower=='below_ballinger'):
   # if (cmin>cmax) and (status_lower=='below_ballinger'):
    if (cmin>cmax):
       # print('red white blue')
        if pos ==0:
            bp =close
            pos=1
            #print('Buying now at'+ str(bp))
            #print(i)
            
        
    elif(cmin<cmax):
        #print('blue white red')
        if pos ==1:
            pos = 0
            sp = close
            #print('Selling now at'+ str(sp))
            #print(i)
            pc = (sp/bp-1)*100
            percentchange.append(pc)
            
    if num ==df['Adj Close'].count()-1 and pos==1:
        pos = 0
        sp = close
        print('Selling now at'+ str(sp))
        pc = (sp/bp-1)*100
        percentchange.append(pc)      
    num +=1

In [ ]:
#### 2. BALLINGER 

In [ ]:
pos = 0
num = 0
percentchange = []

for i in df.index:
    cmin = df['Ema_26'][i]
    cmax = df['Ema_50'][i]
    status_upper = df['status_upper'][i]
    status_lower = df['status_lower'][i]

    
    close = df['Adj Close'][i]
    
    if(status_lower=='below_ballinger'):
       # print('red white blue')
        if pos ==0:
            bp =close
            pos=1
            #print('Buying now at'+ str(bp))
            #print(i)
            
        
    elif(status_upper=='upper_ballinger'):
        #print('blue white red')
        if pos ==1:
            pos = 0
            sp = close
            #print('Selling now at'+ str(sp))
            #print(i)
            pc = (sp/bp-1)*100
            percentchange.append(pc)
            
    if num ==df['Adj Close'].count()-1 and pos==1:
        pos = 0
        sp = close
        #print('Selling now at'+ str(sp))
        pc = (sp/bp-1)*100
        percentchange.append(pc)      
    num +=1
    
#print(percentchange)


In [ ]:
#### 3. STRATEGY BUY CROSSING, SELL BALLINGER

In [ ]:
pos = 0
num = 0
percentchange = []

for i in df.index:
    cmin = df['Ema_26'][i]
    cmax = df['Ema_50'][i]
    status_upper = df['status_upper'][i]
    
    close = df['Adj Close'][i]
    
    #if(status_lower=='below_ballinger'):
   # if (cmin>cmax) and (status_lower=='below_ballinger'):
    if (cmin>cmax):
       # print('red white blue')
        if pos ==0:
            bp =close
            pos=1
            #print('Buying now at'+ str(bp))
            #print(i)
            
        
    elif(status_upper=='upper_ballinger'):
        #print('blue white red')
        if pos ==1:
            pos = 0
            sp = close
            #print('Selling now at'+ str(sp))
            #print(i)
            pc = (sp/bp-1)*100
            percentchange.append(pc)
            
    if num ==df['Adj Close'].count()-1 and pos==1:
        pos = 0
        sp = close
        #print('Selling now at'+ str(sp))
        pc = (sp/bp-1)*100
        percentchange.append(pc)      
    num +=1
    
#print(percentchange)


In [ ]:
#### 4. STRATEGY BUY BALLINGER, SELL CROSSING

In [ ]:
pos = 0
num = 0
percentchange = []

for i in df.index:
    cmin = df['Ema_26'][i]
    cmax = df['Ema_50'][i]
    status_lower = df['status_lower'][i]
    
    close = df['Adj Close'][i]
    
    if(status_lower=='below_ballinger'):
   # if (cmin>cmax) and (status_lower=='below_ballinger'):
       # print('red white blue')
        if pos ==0:
            bp =close
            pos=1
            #print('Buying now at'+ str(bp))
            #print(i)
            
        
    elif(cmin<cmax):
        #print('blue white red')
        if pos ==1:
            pos = 0
            sp = close
            #print('Selling now at'+ str(sp))
            #print(i)
            pc = (sp/bp-1)*100
            percentchange.append(pc)
            
    if num ==df['Adj Close'].count()-1 and pos==1:
        pos = 0
        sp = close
        print('Selling now at'+ str(sp))
        pc = (sp/bp-1)*100
        percentchange.append(pc)      
    num +=1
    
    
#print(percentchange)

In [ ]:
## print results

gains = 0
ng = 0
losses = 0
nl = 0
totallR = 1
for i in percentchange:
    if i >0:
        gains +=i
        ng +=1
    else:
        losses +=i
        nl +=1
    totallR = totallR*((i/100)+1)
totallR = round((totallR-1)*100)
if ng > 0:
    avgGain = gains/ng
    maxR = str(max(percentchange))
else:
    avgGain = 0
    maxR = 'undefined'

if nl>0:
    avgLoss = losses/nl
    maxL = str(min(percentchange))
    ratio = str(-(avgGain/avgLoss))
else:
    avgLoss = 0
    maxL = 'undefined'
    ratio = 'inf'

if ng >0 and nl >0:
    bettingAvg = ng/ng+nl
else:
    bettingAvg = 0
    
    
print()
print('Result for'+stock+"going back to "+str(df.index[0])+" Sample size: "+str(ng+nl)+"trades")
print('EMAs used : ',str(emasUsed))
print('Batting Avg : '+str(bettingAvg))
print('Gain/loss ratio: ' + ratio)
print('Avg gain: ' + str(avgGain))
print('Avg loss: '+ str(avgLoss))
print('Max return: '+ str(maxR))
print('Max loss: ' + str(maxL))
print('Total return over '+str(ng+nl)+' trades: '+str(totallR)+'%')
print()